In [ ]:
import pandas as pd

pickle_path = ""

df = pd.read_pickle(pickle_path)

df.head()

,unpacked_sample_sha1,benign,malicious,packer_name
-386256,7db3ecbe09f60f246a26cc80f6eae59901ad0846,True,False,kkrunchy
-435908,ebbeffe5c5539659f287372703abf860a9afb04e,True,False,kkrunchy
-437497,33ffe28579330c52dd3cd57402321bac1ebbb974,False,True,kkrunchy
-432172,9923bb4f9ee969a44b2491d29d57c2e111970af1,True,False,kkrunchy
-437979,a26f28422341f96299165a423afef4f061b9c561,False,True,kkrunchy


In [2]:
# Filter with non extracted samples
not_extracted_packware = ['upx_a81362b3427e0b8061a937245cff720c32e19a0f.upx.bin', 'upx_9bbfb7e070238e88107b8335190500b6d29bc80d.upx.bin', 'themida-v2_d756de22a8172c1560299ef3146e6c060fba210e.themida-v2.bin', 'themida-v2_b47c2247999b3d45297803313b598b17b544576f.themida-v2.bin', 'telock_7a3a873e5e940c04d11806d2bb874b03d140206f.telock.bin', 'telock_f9fb1623e84b55df84e987f28914756371a5ae37.telock.bin', 'upx_bb7b1bd42a3282a075a0ff61118f4cd0185c9434.upx.bin', 'petite_30058ff28fa1b96e9354285dd2a9eaa617d2d822.petite.bin', 'pecompact_a63e34ec6e7610352cce4cc2eea3ea164eb27bfc.pecompact.bin', 'telock_90dcdae6e283bc401f61ee75c7052ec80a713532.telock.bin']

# Parse the full identifiers to a set for efficient lookup
full_identifiers = set(not_extracted_packware)

# Create a unique identifier in the DataFrame
df['identifier'] = df['packer_name'] + '_' + df['unpacked_sample_sha1'] + '.' + df['packer_name'] + '.bin'

# Filter the DataFrame by checking if the identifier is in the full_identifiers set
df_filtered = df[~df['identifier'].isin(full_identifiers)]

df_filtered

,unpacked_sample_sha1,benign,malicious,packer_name,identifier
-386256,7db3ecbe09f60f246a26cc80f6eae59901ad0846,True,False,kkrunchy,kkrunchy_7db3ecbe09f60f246a26cc80f6eae59901ad0...
-435908,ebbeffe5c5539659f287372703abf860a9afb04e,True,False,kkrunchy,kkrunchy_ebbeffe5c5539659f287372703abf860a9afb...
-437497,33ffe28579330c52dd3cd57402321bac1ebbb974,False,True,kkrunchy,kkrunchy_33ffe28579330c52dd3cd57402321bac1ebbb...
-432172,9923bb4f9ee969a44b2491d29d57c2e111970af1,True,False,kkrunchy,kkrunchy_9923bb4f9ee969a44b2491d29d57c2e111970...
-437979,a26f28422341f96299165a423afef4f061b9c561,False,True,kkrunchy,kkrunchy_a26f28422341f96299165a423afef4f061b9c...
...,...,...,...,...,...
-325448,7d47accafda61a706dd559429a1bf8ec954f55fd,True,False,upx,upx_7d47accafda61a706dd559429a1bf8ec954f55fd.u...
-343940,27f6f8151df889e1006fb2397aa1e4b208dbea2d,False,True,upx,upx_27f6f8151df889e1006fb2397aa1e4b208dbea2d.u...
-360303,a085af29f73e8506b36f5b33edd5c53ca621127f,False,True,upx,upx_a085af29f73e8506b36f5b33edd5c53ca621127f.u...
-357997,7be06c5fd7bb9353f47152852b47d7d547e9e870,False,True,upx,upx_7be06c5fd7bb9353f47152852b47d7d547e9e870.u...


In [3]:
pd.DataFrame({'benign' : df_filtered.groupby(['packer_name'])['benign'].sum(), 'malicious' : df_filtered.groupby(['packer_name'])['malicious'].sum()})

,benign,malicious
packer_name,,
kkrunchy,487,1048
mpress,760,775
obsidium,529,1006
pecompact,237,1297
pelock,716,819
petite,508,1026
telock,205,1327
themida-v2,562,971
upx,498,1034


In [4]:
# Function to sample per packer maintaining distribution with a seed
def sample_packer(df, samples_per_packer=100, random_state=None):
    # Container for the sampled dataframes
    sampled_dfs = []

    # Group by packer name
    grouped = df.groupby('packer_name')

    for name, group in grouped:
        # Check if the group is smaller than the required sample size
        if len(group) < samples_per_packer:
            print(f"Not enough data for {name}, using the available {len(group)} samples.")
            sampled_dfs.append(group)
        else:
            # Calculate the number of benign and malicious samples needed
            benign_count = group['benign'].sum()
            malicious_count = group['malicious'].sum()
            total = benign_count + malicious_count
            benign_fraction = benign_count / total

            # Calculate number of benign and malicious samples to pick
            benign_samples = int(samples_per_packer * benign_fraction)
            malicious_samples = samples_per_packer - benign_samples

            # Sample benign and malicious separately with a seed
            benign_sampled = group[group['benign']].sample(n=benign_samples, replace=False, random_state=random_state)
            malicious_sampled = group[group['malicious']].sample(n=malicious_samples, replace=False, random_state=random_state)

            # Combine the two samples
            combined_sampled = pd.concat([benign_sampled, malicious_sampled])
            sampled_dfs.append(combined_sampled)

    # Concatenate all sampled dataframes
    return pd.concat(sampled_dfs)

# Apply the function with a specific seed
train_df = sample_packer(df_filtered, random_state=17)

train_df.head()

,unpacked_sample_sha1,benign,malicious,packer_name,identifier
-399216,498471614efc2379188b4ab64f12ec4bf9e55439,True,False,kkrunchy,kkrunchy_498471614efc2379188b4ab64f12ec4bf9e55...
-390522,9b7d5c03a1205ee517c0526c1ec0d7c4bf0fe0f8,True,False,kkrunchy,kkrunchy_9b7d5c03a1205ee517c0526c1ec0d7c4bf0fe...
-444840,3b9fc4b68966e069af20229a4b2e08a91f004660,True,False,kkrunchy,kkrunchy_3b9fc4b68966e069af20229a4b2e08a91f004...
-452479,6f7a158263596ec51290ba3de4c4bf4d66b2c193,True,False,kkrunchy,kkrunchy_6f7a158263596ec51290ba3de4c4bf4d66b2c...
-437413,2c7badafb87f1dd680d0685fd90b4c8e628d8be4,True,False,kkrunchy,kkrunchy_2c7badafb87f1dd680d0685fd90b4c8e628d8...


In [5]:
pd.DataFrame({'benign' : train_df.groupby(['packer_name'])['benign'].sum(), 'malicious' : train_df.groupby(['packer_name'])['malicious'].sum()})

,benign,malicious
packer_name,,
kkrunchy,31,69
mpress,49,51
obsidium,34,66
pecompact,15,85
pelock,46,54
petite,33,67
telock,13,87
themida-v2,36,64
upx,32,68


In [6]:
# Filter out rows in df that are in df_sampled based on the identifier
test_df = df_filtered[~df_filtered['identifier'].isin(train_df['identifier'])]

test_df.head()

,unpacked_sample_sha1,benign,malicious,packer_name,identifier
-386256,7db3ecbe09f60f246a26cc80f6eae59901ad0846,True,False,kkrunchy,kkrunchy_7db3ecbe09f60f246a26cc80f6eae59901ad0...
-435908,ebbeffe5c5539659f287372703abf860a9afb04e,True,False,kkrunchy,kkrunchy_ebbeffe5c5539659f287372703abf860a9afb...
-437497,33ffe28579330c52dd3cd57402321bac1ebbb974,False,True,kkrunchy,kkrunchy_33ffe28579330c52dd3cd57402321bac1ebbb...
-432172,9923bb4f9ee969a44b2491d29d57c2e111970af1,True,False,kkrunchy,kkrunchy_9923bb4f9ee969a44b2491d29d57c2e111970...
-437979,a26f28422341f96299165a423afef4f061b9c561,False,True,kkrunchy,kkrunchy_a26f28422341f96299165a423afef4f061b9c...


In [7]:
pd.DataFrame({'benign' : test_df.groupby(['packer_name'])['benign'].sum(), 'malicious' : test_df.groupby(['packer_name'])['malicious'].sum()})

,benign,malicious
packer_name,,
kkrunchy,456,979
mpress,711,724
obsidium,495,940
pecompact,222,1212
pelock,670,765
petite,475,959
telock,192,1240
themida-v2,526,907
upx,466,966


In [ ]:
import os
import shutil

# Destination folder for copied files
destination_folder = "packware/train/100/"

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Copy files
for index, row in train_df.iterrows():
    source_path = "packware/all/" + row['identifier'][:-3] + "xml"
    destination_path = os.path.join(destination_folder, os.path.basename(source_path))

    # Copy file from source to destination
    shutil.copy2(source_path, destination_path)

# Inform user of completion
print("All files have been copied successfully.")

All files have been copied successfully.


In [ ]:
# Destination folder for copied files
destination_folder = "packware/test/"

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Copy files
for index, row in test_df.iterrows():
    source_path = "packware/all/" + row['identifier'][:-3] + "xml"
    destination_path = os.path.join(destination_folder, os.path.basename(source_path))

    # Copy file from source to destination
    shutil.copy2(source_path, destination_path)

# Inform user of completion
print("All files have been copied successfully.")

All files have been copied successfully.


In [9]:
import os
import shutil

# Setup sub-training sets
train_dfs = {}
train_dfs[100] = train_df

# Apply the function with a specific seed
for size in [90,80,70,60,50,40,30,20,10,5,2]:
    train_dfs[size] = sample_packer(train_dfs[min(train_dfs.keys())], samples_per_packer=size, random_state=17)

for size in train_dfs.keys():

    if size == 100:
        continue

    # Destination folder for copied files
    destination_folder = "packware/entire/train/" + str(size) + "/"

    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Copy files
    for index, row in train_dfs[size].iterrows():
        if size == 5:
            source_path = "packware/entire/train/" + str(10) + "/" + row['identifier'][:-3] + "xml"
            destination_path = os.path.join(destination_folder, os.path.basename(source_path))

            # Copy file from source to destination
            shutil.copy2(source_path, destination_path)
        elif size == 2:
            source_path = "packware/entire/train/" + str(5) + "/" + row['identifier'][:-3] + "xml"
            destination_path = os.path.join(destination_folder, os.path.basename(source_path))

            # Copy file from source to destination
            shutil.copy2(source_path, destination_path)

        else:
            source_path = "packware/entire/train/" + str(size+10) + "/" + row['identifier'][:-3] + "xml"
            destination_path = os.path.join(destination_folder, os.path.basename(source_path))

            # Copy file from source to destination
            shutil.copy2(source_path, destination_path)

    # Inform user of completion
    print("All files have been copied successfully.")

All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
All files have been copied successfully.
